In [1]:
from abc import ABC, abstractmethod
import os
import pandas as pd
import geopandas as gpd
import glob
import rasterio
from pathlib import Path
import sdmxthon
from lxml import etree
from tqdm import tqdm
from typing import Callable, Union

In [4]:
land_use = gpd.read_file("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/control_variables/land_use/graffectations.geojson")

In [5]:
land_use.columns

Index(['AFFECTATIO', 'Shape_Leng', 'Shape_Area', 'geometry'], dtype='object')

In [6]:
# calculate area of each land use type
land_use["area"] = land_use["geometry"].area
# rename land use types
land_use["land_use"] = land_use["AFFECTATIO"].replace({
    'Conservation': 'lu_others',
    'Dominante résidentielle': 'lu_residential_community',
    'Industrie': 'lu_commerce_developped',
    'Grande emprise ou grande infrastructure publique': 'lu_commerce_developped',
    'Activités diversifiées': 'lu_commerce_developped',
    'Agricole': 'lu_others',
    'Grand espace vert ou récréation': 'lu_others',
    "Centre-ville d'agglomération": 'lu_commerce_developped'})

In [7]:
land_use

,AFFECTATIO,Shape_Leng,Shape_Area,geometry,area,land_use
0,Conservation,1695.955767,5.352411e+04,"MULTIPOLYGON (((277922.927 5041703.496, 277922...",5.352411e+04,lu_others
1,Conservation,4955.704915,2.815585e+05,"MULTIPOLYGON (((278155.001 5041935.998, 278160...",2.815585e+05,lu_others
2,Conservation,980.766489,1.611968e+04,"MULTIPOLYGON (((279560.597 5041556.912, 279561...",1.611968e+04,lu_others
3,Dominante résidentielle,160392.282573,8.256952e+07,"MULTIPOLYGON (((276676.135 5038836.719, 276676...",8.256952e+07,lu_residential_community
4,Industrie,5692.136199,1.936439e+06,"MULTIPOLYGON (((292906.691 5043390.729, 292916...",1.936439e+06,lu_commerce_developped
...,...,...,...,...,...,...
389,Dominante résidentielle,1349.229272,5.893811e+04,"MULTIPOLYGON (((297128.101 5055785.090, 297140...",5.893811e+04,lu_residential_community
390,Grand espace vert ou récréation,587.929533,1.018238e+04,"MULTIPOLYGON (((300683.000 5058463.184, 300656...",1.018238e+04,lu_others
391,Dominante résidentielle,2097.576949,1.842303e+05,"MULTIPOLYGON (((300915.957 5034660.676, 300928...",1.842303e+05,lu_residential_community
392,Activités diversifiées,1152.570611,5.240486e+04,"MULTIPOLYGON (((298905.419 5036288.560, 298904...",5.240486e+04,lu_commerce_developped


In [8]:
import csv

with open("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/control_variables/population/98-400-X2016003_ENG_CSV/98-400-X2016003_English_CSV_data.csv", "r") as file:
    csv_reader = csv.reader(file)
    rows = []
    for i, row in enumerate(csv_reader):
        if i < 100:
            rows.append(row)
        else:
            break

    column_widths = [max(len(cell) for cell in column) for column in zip(*rows)]
    header = ['Column ' + str(i + 1) for i in range(len(rows[0]))]

    # Print column headers
    for i, width in enumerate(column_widths):
        print(header[i].ljust(width), end=' | ')
    print()

    # Print grid separator
    for width in column_widths:
        print('-' * width, end=' | ')
    print()

    # Print rows
    for row in rows:
        for i, width in enumerate(column_widths):
            print(row[i].ljust(width), end=' | ')
        print()


Column 1       | Column 2       | Column 3  | Column 4 | Column 5 | Column 6          | Column 7      | Column 8     | Column 9                                         | Column 10                                              | Column 11                                          | Column 12                                 | Column 13                          | Column 14                            | 
-------------- | -------------- | --------- | -------- | --- | ----------------- | ------------- | ------------ | ------------------------------------------------ | ------------------------------------------------------ | -------------------------------------------------- | ----------------------------------------- | ---------------------------------- | ------------------------------------ | 
﻿"CENSUS_YEAR" | GEO_CODE (POR) | GEO_LEVEL | GEO_NAME | GNR | DATA_QUALITY_FLAG | CSD_TYPE_NAME | ALT_GEO_CODE | DIM: Age (in single years) and average age (127) | Member ID: Age (in single years) and a

In [2]:
count_station_year = pd.read_csv("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/count_station_year.csv")
count_station = pd.read_csv("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/count_station.csv")
# join count_station_year with count_station
count_station_year = count_station_year.merge(count_station[["count_point_id", "latitude", "longitude"]], on="count_point_id", how="left")
# convert count_station_year to geodataframe
count_station_year = gpd.GeoDataFrame(count_station_year, geometry=gpd.points_from_xy(count_station_year.longitude, count_station_year.latitude)).\
    set_crs(epsg=4326)

In [12]:
# join count_station_year with count_station
# take 500m from the count station
utm_crs = count_station_year.estimate_utm_crs()
count_station_year_utm = count_station_year.to_crs(utm_crs)
count_station_year_utm["geometry"] = count_station_year_utm.buffer(500)
land_use = land_use.to_crs(utm_crs)
# spatial join count_station_year_utm with landuse_2016
count_land_use = gpd.sjoin(count_station_year_utm, land_use,
                            how="left", op="intersects").reset_index()

# group by count_point_id, year, and land_use
count_land_use = count_land_use.groupby(["count_point_id", "year", "land_use"], as_index=False).\
    agg({"area": "sum"})
    
# pivot table
count_land_use = count_land_use.pivot_table(index=["count_point_id", "year"], columns="land_use", values="area").reset_index()

# fill NaN with 0
count_land_use = count_land_use.fillna(0)

# calculate total area and divide land use area by total area
count_land_use["total_area"] = count_land_use.iloc[:, 2:].sum(axis=1)
count_land_use.iloc[:, 2:] = count_land_use.iloc[:, 2:].div(count_land_use["total_area"], axis=0)
count_land_use = count_land_use.drop(columns="total_area")
count_land_use

/Users/koichiito/opt/anaconda3/envs/bike_svi_arm/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


land_use,count_point_id,year,lu_commerce_developped,lu_others,lu_residential_community
0,725.0,2008,0.741401,0.000000,0.258599
1,726.0,2008,0.736189,0.000000,0.263811
2,728.0,2008,0.976817,0.000000,0.023183
3,729.0,2008,0.016281,0.023038,0.960680
4,731.0,2008,0.016281,0.023038,0.960680
...,...,...,...,...,...
5666,10125.0,2023,0.000000,0.000000,1.000000
5667,10126.0,2023,0.096337,0.000000,0.903663
5668,10127.0,2023,0.000000,0.008528,0.991472
5669,10128.0,2023,0.000000,0.000000,1.000000


In [11]:
count_land_use

land_use,count_point_id,year,lu_commerce_developped,lu_others,lu_residential_community
0,725.0,2008,2.616135e+07,0.000000e+00,9.125027e+06
1,726.0,2008,2.546425e+07,0.000000e+00,9.125027e+06
2,728.0,2008,6.460641e+06,0.000000e+00,1.533320e+05
3,729.0,2008,1.411764e+06,1.997697e+06,8.330193e+07
4,731.0,2008,1.411764e+06,1.997697e+06,8.330193e+07
...,...,...,...,...,...
5666,10125.0,2023,0.000000e+00,0.000000e+00,8.330193e+07
5667,10126.0,2023,8.880540e+06,0.000000e+00,8.330193e+07
5668,10127.0,2023,0.000000e+00,7.165197e+05,8.330193e+07
5669,10128.0,2023,0.000000e+00,0.000000e+00,4.335544e+07


In [8]:
import concurrent.futures
import pandas as pd
from tqdm import tqdm
from ohsome import OhsomeClient
from tenacity import retry, stop_after_attempt

@retry(stop=stop_after_attempt(3))
def fetch_poi_data(client, bcircles, year):
    return client.elements.count.post(
        bcircles=bcircles,
        time=f"{str(year)}-01-01",
        filter="amenity=* and type:node"
    )

def fetch_poi(row, buffer, client):
    result = {}
    try:
        response = fetch_poi_data(client, [row.longitude, row.latitude, buffer], row.year)
        response_df = response.as_dataframe().reset_index()
        poi_num = response_df["value"][0]
        result["poi"] = poi_num
    except Exception as e: 
        print(e)
    return result

def retrieve_poi(count_station, buffer = 500, output_folder="."):
    client = OhsomeClient()
    print("Ohsome client has been initialized")
    print(f"Earliest date available is {client.start_timestamp} and latest is {client.end_timestamp}")
    tqdm.pandas()
    
    results = {}
    # Using ThreadPoolExecutor to parallelize the process
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(fetch_poi, row, buffer, client): row for row in count_station.itertuples()}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            row = futures[future]
            try:
                data = future.result()
                results[row.Index] = data
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    # Convert results to DataFrame and save as csv
    df = pd.DataFrame.from_dict(results, orient='index')
    df.to_csv(os.path.join(output_folder, "poi.csv"), index=False)


In [9]:
# just for testing: use the first 10 rows
df = retrieve_poi(count_station_year[:100], buffer=500, output_folder=".")

Ohsome client has been initialized
Earliest date available is 2007-10-08T00:00:00Z and latest is 2023-06-04T20:00Z


100%|██████████| 100/100 [00:21<00:00,  4.76it/s]


In [21]:
import os
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
import richdem as rd
import concurrent.futures
from shapely.geometry import Point

# load data ---------------------------------------------------------------
root_dir = "/Volumes/ExFAT/bike_svi"
dem_path = os.path.join(root_dir, "data/external/cities/London/gis_variables/slope/LIDAR_10m_DTM_Composite_2019/LIDAR_10m_DTM_Composite.tif")
count_station_path = os.path.join(root_dir, "data/external/cities/London/count_station.csv")

dem = rasterio.open(dem_path)
count_station = pd.read_csv(count_station_path)

# create geopandas GeoDataFrame
geometry = [Point(xy) for xy in zip(count_station['longitude'], count_station['latitude'])]
count_station = gpd.GeoDataFrame(count_station, geometry=geometry)
count_station = count_station.set_crs("EPSG:4326")
count_station = count_station.to_crs("EPSG:3857")
count_station['geometry'] = count_station.geometry.buffer(500)

# compute slope -----------------------------------------------------------
dem_arr = dem.read(1)
rd_arr = rd.rdarray(dem_arr, no_data=dem.nodata)
slope = rd.TerrainAttribute(rd_arr, attrib='slope_riserun')

# Define function to calculate mean slope for a station
def calculate_mean_slope(geom):
    with rasterio.open(dem_path) as src:
        out_image, _ = rasterio.mask.mask(src, [geom], crop=True)
        out_image = rd.rdarray(out_image[0], no_data=src.nodata)
        slope = rd.TerrainAttribute(out_image, attrib='slope_riserun')
    return np.nanmean(slope)

# Calculate mean slope for each station in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    slope_means = list(executor.map(calculate_mean_slope, count_station.geometry))

count_station['slope'] = slope_means

# save --------------------------------------------------------------------
count_station = count_station.drop(columns=['geometry'])
count_station.to_csv(os.path.join(root_dir, "data/processed/cities/London/slope.csv"), index=False)


poi    6.0
dtype: float64

In [10]:
import numpy as np
from osgeo import gdal
from tqdm import tqdm

def calculate_slope(DEM):
    """Calculate slope of DEM using Horn's method."""
    # Padded DEM for boundary conditions
    padded_DEM = np.pad(DEM, ((1, 1), (1, 1)), mode='edge')

    # Calculate dz/dx and dz/dy
    dzdx = (padded_DEM[2:, 1:-1] - padded_DEM[:-2, 1:-1] +
            2*(padded_DEM[2:, 2:] - padded_DEM[:-2, 2:]) +
            padded_DEM[2:, :-2] - padded_DEM[:-2, :-2]) / 8

    dzdy = (padded_DEM[1:-1, 2:] - padded_DEM[1:-1, :-2] +
            2*(padded_DEM[2:, 2:] - padded_DEM[:-2, 2:]) +
            padded_DEM[:-2, 2:] - padded_DEM[:-2, :-2]) / 8

    # Calculate slope
    slope = np.arctan(np.sqrt(dzdx**2 + dzdy**2)) * 180 / np.pi
    return slope

# Open the DEM
ds = gdal.Open("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/gis_variables/slope/cdem_dem_031H_tif/cdem_dem_031H.tif")
band = ds.GetRasterBand(1)

# Convert to numpy array
DEM = band.ReadAsArray().astype(float)

# Calculate the slope
slope = calculate_slope(DEM)

# Save the slope array as a new GeoTIFF
driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create("/Volumes/ExFAT2/bike_svi/data/external/cities/Montreal/gis_variables/slope/cdem_dem_031H_tif/slope.tif", DEM.shape[1], DEM.shape[0], 1, gdal.GDT_Float32)
outdata.SetGeoTransform(ds.GetGeoTransform())
outdata.SetProjection(ds.GetProjection())
outdata.GetRasterBand(1).WriteArray(slope)
outdata.GetRasterBand(1).SetNoDataValue(-9999)
outdata.FlushCache()
outdata = None
